# MobileNet 모델로 데이터셋에서 특성 추출하기
* 21XXXX_01_video-to-numpy-save.ipynb, 21XXXX_02_create-numpy-... .ipynb 파일에서 이어지는 코드입니다.
* **`data_training_ar`**, **`data_test_ar`** 등을 불러와 **`X_train`**, **`y_train`** 을 만듭니다.

# Imports 

In [1]:
import numpy as np # numpy 배열
import os # 파일 및 폴더의 경로 지정을 위한 모듈
import tensorflow as tf # 텐서플로우
from tensorflow import keras # 케라스
import time #프로세스 소요시간 표시 목적
import random

# 데이터 불러오기 : data_training_ar, data_test_ar
* **`data_training_ar`**, **`data_test_ar`** : (동영상 개수, 프레임 수, 이미지 크기, 이미지 크기, RGB)

In [2]:
data_training_ar=np.load('D:/datasets/AllVideo_numpy_list_pickle/02_data_training_Numpy_210507.npy') 
data_training_ar.shape #> (2865, 30, 160, 160, 3) 

(2865, 30, 160, 160, 3)

In [3]:
data_test_ar=np.load('D:/datasets/AllVideo_numpy_list_pickle/02_data_test_Numpy_210507.npy') 
data_test_ar.shape #> (717, 30, 160, 160, 3)

(717, 30, 160, 160, 3)

# 데이터 reshape

In [4]:
data_training_ar=data_training_ar.reshape(data_training_ar.shape[0]*30, 160, 160, 3) 
data_training_ar.shape #> (85950, 160, 160, 3)

(85950, 160, 160, 3)

In [5]:
data_test_ar=data_test_ar.reshape(data_test_ar.shape[0]*30, 160, 160, 3) 
data_test_ar.shape #> (21510, 160, 160, 3)

(21510, 160, 160, 3)

# 베이스모델 생성 : MobileNet

In [6]:
base_model=keras.applications.mobilenet.MobileNet(input_shape=(160, 160, 3),
                                                  include_top=False,
                                                  weights='imagenet', classes=2)

In [7]:
base_model.summary() #> (None, 5, 5, 1024)

Model: "mobilenet_1.00_160"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 80, 80, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 80, 80, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 80, 80, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 80, 80, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 80, 80, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 80, 80, 32) 

# Xception에 각 프레임 이미지 투입, 변환 : predict
* **`Xeption_X_train`**, **`Xeption_X_test`** 생성

In [8]:
np.random.seed(42)

In [9]:
X_train=base_model.predict(data_training_ar) #> 여기서 매우 큰 메모리 부족을 겪는다.
X_train.shape #> (85950, 5, 5, 1024)

(85950, 5, 5, 1024)

In [10]:
X_test=base_model.predict(data_test_ar)
X_test.shape #> (21510, 5, 5, 1024)

(21510, 5, 5, 1024)

In [11]:
#np.save('D:/datasets/AllVideo_numpy_list_pickle/Xception_x_train.npy', X_train)

In [12]:
#np.save('D:/datasets/AllVideo_numpy_list_pickle/Xception_x_test.npy', X_test)

# LSTM에 투입할 수 있게끔 MobileNet를 통과한 결과물을 reshape
* LSTM 모델에 투입하여 훈련시키려면 : (파일개수, 프레임 수(=30), 25600) 형태의 ndarray 배열이어야 합니다.

In [13]:
# reshaping the training as well as validation frames in single dimension
X_train_reshaped = X_train.reshape(int(X_train.shape[0]/30), 30, 5*5*1024) #> (2865, 30, 25600) ndarray

In [14]:
X_test_reshaped = X_test.reshape(int(X_test.shape[0]/30), 30, 5*5*1024) #> (717, 30, 25600) ndarray

In [15]:
X_train_reshaped.shape, X_test_reshaped.shape

((2865, 30, 25600), (717, 30, 25600))

In [16]:
np.save('D:/datasets/AllVideo_numpy_list_pickle/MobileNet_x_train_reshaped.npy', X_train_reshaped)

In [17]:
np.save('D:/datasets/AllVideo_numpy_list_pickle/MobileNet_x_test_reshaped.npy', X_test_reshaped)